# REMoDNaV Error Analysis

In [1]:
import os
from typing import Literal, List

import numpy as np
import pandas as pd
from matplotlib.pyplot import colorbar
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.io as pio

import peyes

import analysis.utils as u
import analysis._article_results.lund2013._helpers as lund
import analysis._article_results.hfc._helpers as hfc
import peyes._utils.constants

pio.renderers.default = "notebook"

C:\Users\jonathanni\PycharmProjects\pEYES\venv\Lib\site-packages\statsmodels\tools\tools.py:6: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 2.3.0)
  import scipy.linalg


### Set Constants

In [2]:
GT_ANNOTATORS = ["RA", "MN"]  # ground truth annotators
ONSET_STR, OFFSET_STR = peyes.constants.ONSET_STR, peyes.constants.OFFSET_STR

# visualization constants
GRID_LINE_COLOR, GRID_LINE_WIDTH = "lightgray", 1
ZERO_LINE_WIDTH = 2 * GRID_LINE_WIDTH

SINGLE_MEASURE_OPACITY, MEDIAN_OPACITY = 0.2, 1.0
SINGLE_MEASURE_LINE = dict(width=2 * GRID_LINE_WIDTH)
SINGLE_MEASURE_MARKER = dict(symbol="circle-open", size=6 * GRID_LINE_WIDTH,)
MEADIAN_LINE = dict(width=SINGLE_MEASURE_LINE['width'])
MEADIAN_MARKER = dict(symbol="circle", size=2 * SINGLE_MEASURE_MARKER['size'],)

FONT_FAMILY, FONT_COLOR = "Calibri", "black"
TITLE_FONT = dict(family=FONT_FAMILY, size=25, color=FONT_COLOR)
SUBTITLE_FONT = dict(family=FONT_FAMILY, size=20, color=FONT_COLOR)
AXIS_LABEL_FONT = dict(family=FONT_FAMILY, size=20, color=FONT_COLOR)
AXIS_TICK_FONT = dict(family=FONT_FAMILY, size=18, color=FONT_COLOR)
AXIS_LABEL_STANDOFF = 3

# output dir
FIGURES_DIR = os.path.join(lund.PROCESSED_DATA_DIR, "supplementary_M")

## Load Data

In [3]:
def load_labels(
        dataset_name: Literal["lund", "hfc"], labeler_subset: List[str] = None
) -> pd.DataFrame:
    if dataset_name == "lund":
        dataset_name = lund.DATASET_NAME
        directory = lund.PROCESSED_DATA_DIR
        stim_type = lund.STIMULUS_TYPE
        file_name = peyes.constants.LABELS_STR
    elif dataset_name == "hfc":
        dataset_name = hfc.DATASET_NAME
        directory = hfc.PROCESSED_DATA_DIR
        stim_type = hfc.STIMULUS_TYPE
        file_name = f"raw_{peyes.constants.LABELS_STR}"
    else:
        raise ValueError(f"Unknown dataset name: {dataset_name}")
    all = pd.read_pickle(os.path.join(directory, dataset_name, file_name + ".pkl"))
    iter1 = all.xs(1, level=peyes.constants.ITERATION_STR, axis=1)
    stim_trials = u.get_trials_for_stimulus_type(dataset_name, stim_type)
    stim_data = iter1.loc[:, stim_trials]

    labeler_subset = labeler_subset or ["RA", "MN", "engbert", "remodnav"]
    is_labeler_subset = np.isin(stim_data.columns.get_level_values(peyes.constants.LABELER_STR), labeler_subset)
    stim_data = stim_data.loc[:, is_labeler_subset]
    return stim_data

In [4]:
lund_labels = load_labels("lund",)
hfc_labels = load_labels("hfc")

## Label Error Analysis
#### (0) Visualization Helper Functions

In [5]:
def _figure_labels(lbl: str) -> str:
    parsed = peyes.parse_label(lbl).name
    if parsed in {"FIXATION", "SACCADE", "PSO", "BLINK"}:
        return parsed[:3]
    if parsed == "SMOOTH_PURSUIT":
        return "SP"
    if parsed == "UNDEFINED":
        return "U/I"
    raise ValueError(f"Unrecognized label: {parsed}")


def _add_distribution_bars(fig: go.Figure, df: pd.DataFrame, col_idx: int, showlegend: bool) -> go.Figure:
    df[peyes.constants.LABEL_STR] = df[peyes.constants.LABEL_STR].map(lambda lbl: peyes.parse_label(lbl))
    df[peyes.constants.LABELER_STR] = df[peyes.constants.LABELER_STR].map(
        lambda lblr: "REMoDNaV" if lblr == "remodnav" else f"Ann. {lblr}" if lblr in ["MN", "RA"] else lblr.upper()
    )
    sorted_labels = sorted(     # make sure the "Undefined" label is last
        df[peyes.constants.LABEL_STR].unique(), key=lambda l: 100 if l==0 else l
    )
    for lbl in sorted_labels:
        sub_data = df[df[peyes.constants.LABEL_STR] == lbl]
        label_name = _figure_labels(lbl)
        fig.add_trace(
            go.Bar(
                x=sub_data[peyes.constants.LABELER_STR],
                y=sub_data["percentage"],
                name=label_name,
                legendgroup=label_name,
                offsetgroup=col_idx,
                marker_color=peyes._utils.visualization_utils._DEFAULT_COLORMAP[int(peyes.parse_label(lbl))],
                hovertemplate=f"{label_name}: %{{y:.2f}}%",
                showlegend=showlegend,
            ),
            row=1, col=col_idx + 1,
        )
    return fig


def _finalize_figure(fig: go.Figure, title: str, yaxis_label: str) -> go.Figure:
    fig.update_annotations(font=SUBTITLE_FONT, yref="paper", yanchor="bottom")
    fig.update_xaxes(
        # title=dict(text=peyes.constants.LABELER_STR.title(), font=AXIS_LABEL_FONT, standoff=10),
        showline=False,
        showgrid=False, gridcolor=GRID_LINE_COLOR, gridwidth=GRID_LINE_WIDTH,
        zeroline=False, zerolinecolor=GRID_LINE_COLOR, zerolinewidth=ZERO_LINE_WIDTH,
        tickfont=AXIS_TICK_FONT,
    )
    fig.update_yaxes(
        title=dict(font=AXIS_LABEL_FONT, standoff=AXIS_LABEL_STANDOFF),
        showline=False,
        showgrid=True, gridcolor=GRID_LINE_COLOR, gridwidth=GRID_LINE_WIDTH,
        zeroline=True, zerolinecolor=GRID_LINE_COLOR, zerolinewidth=ZERO_LINE_WIDTH,
        tickfont=AXIS_TICK_FONT,
    )
    fig.update_yaxes(title_text=yaxis_label, row=1, col=1)
    fig.update_layout(
        barmode='stack',
        title=dict(text=title, font=TITLE_FONT),
        width=1000, height=400,
        plot_bgcolor='rgba(0, 0, 0, 0)',
        # paper_bgcolor='rgba(0, 0, 0, 0)',
        margin=dict(l=0, r=0, t=45, b=0, pad=0),
        legend=dict(font=AXIS_TICK_FONT),
    )
    return fig

### (1) Label Distribution
We visualize the percentage of each label in the dataset, grouped by labeler.

In [26]:
def _calc_label_distribution(data: pd.DataFrame, labelers: list[str]) -> pd.DataFrame:
    def calc_single_labeler_dist(data: pd.DataFrame, labeler: str) -> pd.Series:
        percentages = (
            data
            .xs(labeler, level=peyes.constants.LABELER_STR, axis=1)
            .stack(future_stack=True)
            .value_counts()
            .sort_index()
        )
        percentages /= percentages.sum()
        percentages *= 100
        percentages.index = percentages.index.map(peyes.parse_label)
        return percentages

    df = pd.concat([calc_single_labeler_dist(data, lblr).rename(lblr) for lblr in labelers], axis=1).fillna(0)
    df = df.sort_index().stack(future_stack=True).reset_index()
    df.columns = [peyes.constants.LABEL_STR, peyes.constants.LABELER_STR, "percentage"]
    return df


column_titles = ["<b><i>lund2013</i></b>", "<b><i>HFC</i></b>"]
label_dist_fig = make_subplots(
    rows=1, cols=len(column_titles), column_titles=column_titles,
    shared_xaxes=False, shared_yaxes=True, x_title="Labeler",
    horizontal_spacing=0.025,
)

for c, ds in enumerate(column_titles):
    data = lund_labels if c == 0 else hfc_labels
    # Either calculate all labeler distributions...
    labelers = u.sort_labelers(data.columns.get_level_values(peyes.constants.LABELER_STR).unique())
    df = _calc_label_distribution(data, labelers)
    fig = _add_distribution_bars(label_dist_fig, df, col_idx=c, showlegend=(c == 0))
label_dist_fig = _finalize_figure(label_dist_fig, title="Event Type Distribution", yaxis_label="Label Distribution (%)")

label_dist_fig.for_each_annotation(lambda ann: ann.update(
    font=AXIS_LABEL_FONT,
    y=-0.05 if ann.text=="Labeler" else 0.95
))
label_dist_fig.update_xaxes(
    title=dict(font=AXIS_LABEL_FONT),
    tickfont={**AXIS_TICK_FONT, **dict(size=16)},
)
label_dist_fig.update_layout(
    legend=dict(
        font=dict(size=16)
    ),
    margin=dict(l=0, r=0, t=40, b=40, pad=0),
    plot_bgcolor='rgba(0, 0, 0, 0)',
    paper_bgcolor='rgba(0, 0, 0, 0)',
)
peyes.visualize.save_figure(
    go.Figure(label_dist_fig).update_layout(title=None, margin=dict(t=10),),
    "M1", FIGURES_DIR, as_png=True, as_json=True
)
label_dist_fig.show()

### (2) REMoDNaV Confusion
Calculate the confusion matrix of detected REMoDNaV labels, to estimate the source of erroneous labels.

In [7]:
def _calculate_confusion_matrix(
        data: pd.DataFrame,
        gt_annotator: str,
        pred_annotator: str,
        normalize: bool,
) -> pd.DataFrame:
    gt = data.xs(gt_annotator, level=peyes.constants.LABELER_STR, axis=1).stack(future_stack=True).rename(gt_annotator)
    pred = data.xs(pred_annotator, level=peyes.constants.LABELER_STR, axis=1).stack(future_stack=True).rename(pred_annotator)
    if normalize:
        res = pd.crosstab(gt, pred, normalize="columns") * 100
    else:
        res = pd.crosstab(gt, pred, margins=True)
    res = (
        res
        .sort_index(key=lambda idx: idx.map(lambda l: 100 if l==0 else l))      # make sure "Undefined" is last row
        .loc[:, sorted(res.columns, key=lambda l: 100 if l==0 else l)]          # make sure "Undefined" is last col
        .rename(
            columns=lambda lbl: lbl if str(lbl).lower() == "all" else peyes.parse_label(lbl),
            index=lambda lbl: lbl if str(lbl).lower() == "all" else peyes.parse_label(lbl),
        )
    )
    return res

In [8]:
print("RA-REMoDNaV Confusion Matrix")
display(_calculate_confusion_matrix(lund_labels, "RA", "remodnav", True))

print("MN-REMoDNaV Confusion Matrix")
display(_calculate_confusion_matrix(lund_labels, "MN", "remodnav", True))

RA-REMoDNaV Confusion Matrix


remodnav,1,2,3,4,5,0
RA,,,,,,
1,95.332249,11.519165,25.251960,76.931442,0.0,32.679739
2,0.180283,75.697371,7.670773,0.709220,0.0,0.000000
3,1.330564,10.947221,63.073908,0.661939,0.0,3.267974
4,2.876683,1.113787,3.303471,11.806147,0.0,0.000000
5,0.272383,0.050171,0.000000,9.820331,100.0,60.566449
0,0.007838,0.672286,0.699888,0.070922,0.0,3.485839


MN-REMoDNaV Confusion Matrix


remodnav,1,2,3,4,5,0
MN,,,,,,
1,97.512025,11.194896,27.863436,86.307415,0.0,32.832080
2,0.149278,75.522042,4.588840,0.615327,0.0,0.000000
3,1.354564,12.703016,66.813510,0.944745,0.0,2.506266
4,0.588821,0.145012,0.256975,1.957859,0.0,0.000000
5,0.395312,0.188515,0.293686,9.522034,100.0,64.661654
0,0.000000,0.246520,0.183554,0.652620,0.0,0.000000


In [23]:
confusion_fig = make_subplots(
    rows=1, cols=len(GT_ANNOTATORS),
    # column_titles=list(map(lambda gt: f"Ann. <i>{gt}</i>", GT_ANNOTATORS)),
    shared_xaxes=False, shared_yaxes=True, x_title="REMoDNaV Label",
    # y_title="Annotator Label",
    horizontal_spacing=0.05,
)
for c, gt in enumerate(GT_ANNOTATORS):
    confusion_data = _calculate_confusion_matrix(lund_labels, gt, "remodnav", normalize=True)
    confusion_fig.add_trace(go.Heatmap(
        x=confusion_data.columns.map(_figure_labels),
        y=confusion_data.index.map(_figure_labels),
        z=confusion_data.values,
        text=confusion_data.map(lambda val: f"{val:.1f}%"),
        texttemplate="%{text}", textfont={**AXIS_TICK_FONT, **dict(size=14)},
        colorbar=dict(tickfont={**AXIS_TICK_FONT, **dict(size=14)}), colorscale="Reds", showscale=c==0,
    ), row=1, col=c + 1)
    confusion_fig.update_xaxes(
        row=1, col=c + 1,
        tickfont={**AXIS_TICK_FONT, **dict(size=16)},
        showgrid=False, gridcolor=GRID_LINE_COLOR, gridwidth=GRID_LINE_WIDTH,
        zeroline=False, zerolinecolor=GRID_LINE_COLOR, zerolinewidth=ZERO_LINE_WIDTH,
    )
    confusion_fig.update_yaxes(
        row=1, col=c+1,
        autorange="reversed",
        tickfont={**AXIS_TICK_FONT, **dict(size=16)},
        title=dict(text=f"Ann. {gt}", font=AXIS_LABEL_FONT, standoff=AXIS_LABEL_STANDOFF),
        showgrid=False, gridcolor=GRID_LINE_COLOR, gridwidth=GRID_LINE_WIDTH,
        zeroline=False, zerolinecolor=GRID_LINE_COLOR, zerolinewidth=ZERO_LINE_WIDTH,
    )

confusion_fig.update_annotations(
    font=AXIS_LABEL_FONT, y=0.03
)
confusion_fig.update_layout(
    width=1000, height=400,
    title=dict(text="REMoDNaV Confusion Matrix (<i>lund2013<sup>+</sup>-image</i>)", font=TITLE_FONT),
    plot_bgcolor='rgba(0, 0, 0, 0)',
    paper_bgcolor='rgba(0, 0, 0, 0)',
    margin=dict(l=50, r=0, t=40, b=50, pad=0),
)

peyes.visualize.save_figure(
    go.Figure(confusion_fig).update_layout(title=None, margin=dict(t=5),),
    "M2", FIGURES_DIR, as_png=True, as_json=True
)
confusion_fig.show()

### (3) REMoDNaV SP Misclassification Figure
The confusion matrices above show that the largest error rate (off-diagonal values) occurs when REMoDNaV misclassifies _FIXATION_ as _SP_. Here we show the distribution of GT labels corresponding to REMoDNaV's _SP_ classification.

In [10]:
def _calc_sp_misclassifications(data: pd.DataFrame) -> pd.DataFrame:
    remodnav_labels = data.xs("remodnav", level=peyes.constants.LABELER_STR, axis=1)
    sp_indices = remodnav_labels[remodnav_labels == peyes.parse_label("smooth_pursuit")].stack().index
    data_on_sp = data.stack(peyes.constants.TRIAL_ID_STR, future_stack=True).loc[sp_indices]
    counts = pd.concat([
        data_on_sp[col].value_counts().rename(col) for col in GT_ANNOTATORS
    ], axis=1).fillna(0).sort_index()
    counts.columns = [f"Ann. {col}" for col in counts.columns]
    percentages = 100 * counts / counts.sum()
    df = percentages.stack(future_stack=True).reset_index()
    df.columns = [peyes.constants.LABEL_STR, peyes.constants.LABELER_STR, "percentage"]
    return df


column_titles = ["<b><i>lund2013</i></b>", "<b><i>HFC</i></b>"]
fig = make_subplots(
    rows=1, cols=len(column_titles), column_titles=column_titles,
    shared_xaxes=False, shared_yaxes=True,
    horizontal_spacing=0.025,
)

for c, ds in enumerate(column_titles):
    data = lund_labels if c == 0 else hfc_labels
    # Either calculate all labeler distributions...
    labelers = u.sort_labelers(data.columns.get_level_values(peyes.constants.LABELER_STR).unique())
    df = _calc_sp_misclassifications(data)
    fig = _add_distribution_bars(fig, df, col_idx=c, showlegend=(c == 0))
fig = _finalize_figure(fig, title="REModNaV SP Misclassifications", yaxis_label="Confused Label (%)").update_layout(
    width=750, height=350
)
fig.show()

## Fixation-Oriented REMoDNaV
We replace REMoDNaV's misclassified _SP_ labels with _fixation_ labels, and analyze the results with respect to the original GT labels.

In [11]:
import scipy.stats as stats

import analysis.process.sample_metrics as samp
import analysis.process.temporal_alignment as temp

from analysis._article_results.lund2013._helpers import LABELER_PLOTTING_CONFIG as lund_plotting_config
from analysis._article_results.hfc._helpers import LABELER_PLOTTING_CONFIG as hfc_plotting_config

LABELERS = GT_ANNOTATORS + ["engbert", "remodnav", "remodnav_new"]

In [12]:
def _replace_label(data: pd.DataFrame, labeler: str, old_label, new_label) -> pd.DataFrame:
    new_data = (
        data
        .copy(deep=True)
        .stack(peyes.constants.TRIAL_ID_STR, future_stack=True)
    )
    new_data[f"{labeler}_new"] = new_data[labeler].replace(peyes.parse_label(old_label), peyes.parse_label(new_label))
    new_data = new_data.unstack(peyes.constants.TRIAL_ID_STR)
    new_data.columns = new_data.columns.reorder_levels(data.columns.names)

    # add "iteration" level to the columns, with value 1 across all columns
    new_data[peyes.constants.ITERATION_STR] = 1
    new_data = new_data.set_index(peyes.constants.ITERATION_STR, append=True).unstack(peyes.constants.ITERATION_STR)
    return new_data

lund_labels2 = _replace_label(lund_labels, "remodnav", "smooth_pursuit", "fixation")
hfc_labels2 = _replace_label(hfc_labels, "remodnav", "smooth_pursuit", "fixation")
# replace non-fixation labels with 'undefined' label as HFC only includes fixation labels
hfc_labels2 = hfc_labels2.replace({
    lbl: peyes.parse_label("undef", safe=True) for lbl in np.unique(hfc_labels2.values)
    if lbl != peyes.parse_label("fixation", safe=False) or pd.isna(lbl)
})

In [13]:
dataset_names = ["<b><i>lund2013</i></b>", "<b><i>HFC</i></b>"]
sample_metrics = {"cohen's_kappa": "<i>Cohen's Kappa</i>", "mcc": "<i>MCC</i>", "complement_nld": "<i>1-NLD</i>"}
figures = {
    gt: make_subplots(
        rows=len(sample_metrics), cols=len(dataset_names),
        row_titles=list(sample_metrics.values()), column_titles=dataset_names,
        shared_xaxes=True, shared_yaxes=True,
        horizontal_spacing=0.02, vertical_spacing=0.04,
    ) for gt in GT_ANNOTATORS
}

for c, ds in enumerate(dataset_names):
    if c == 0:
        dataset = lund_labels2
        plotting_config = lund_plotting_config
    else:
        dataset = hfc_labels2
        plotting_config = hfc_plotting_config
    data = samp.calculate_global_sample_metrics(dataset, GT_ANNOTATORS)
    for r, (metric_key, metric_name) in enumerate(sample_metrics.items()):
        for gt in GT_ANNOTATORS:
            curr_fig = figures[gt]
            for pred in LABELERS:
                if pred == gt:
                    continue  # skip the ground truth annotator
                if pred in GT_ANNOTATORS:
                    pred_name = f"Ann. {pred}"
                    pred_color = plotting_config["Other Human"][1]
                elif pred == "remodnav":
                    pred_name = "REMoDNaV"
                    pred_color = plotting_config[pred][1]
                elif pred == "remodnav_new":
                    pred_name = "REMoDNaV (No SP)"
                    pred_color = "#DA9600"
                else:
                    pred_name = pred.upper()  # e.g., "engbert"
                    pred_color = plotting_config[pred][1]
                subset = (
                    data
                    .xs(1, level=peyes.constants.ITERATION_STR, axis=1)
                    .xs(gt, level=u.GT_STR, axis=1)
                    .xs(pred, level=u.PRED_STR, axis=1)
                    .xs(metric_key, axis=0)
                )
                if subset.empty:
                    continue
                curr_fig.add_trace(
                    row=r + 1, col=c + 1,
                    trace=go.Violin(
                        x=subset.values, y0=pred_name, name=pred, legendgroup=pred, scalegroup=metric_name,
                        line_color="black", fillcolor=pred_color,
                        box=dict(visible=False, width=0.95, line=dict(width=1)),
                        meanline=dict(visible=True, width=3, color=GRID_LINE_COLOR),
                        side="positive", width=1.8, points=False, opacity=1, spanmode="hard",
                        showlegend=(c == 0 and r == 0), visible=True,
                    )
                )

# update figure layouts
for gt, fig in figures.items():
    for ann in fig.layout.annotations:
        ann.textangle = 0
        ann.xref = ann.yref = "paper"
        ann.xanchor = "center"
        ann.yanchor = "bottom"
        if ann.text in dataset_names:
            ann.font = SUBTITLE_FONT
            ann.y = 1.01
            continue
        if ann.text in sample_metrics.values():
            ann.font = AXIS_LABEL_FONT
            idx = list(sample_metrics.values()).index(ann.text)
            ann.y = 0.635 if idx == 0 else 0.3 if idx == 1 else -0.1
            ann.x = 0.5
    fig.update_xaxes(
        showline=False,
        showgrid=False, gridcolor=GRID_LINE_COLOR, gridwidth=GRID_LINE_WIDTH,
        zeroline=False, zerolinecolor=GRID_LINE_COLOR, zerolinewidth=ZERO_LINE_WIDTH,
        tickfont=AXIS_TICK_FONT,
        range=[-0.02, 1.02]
    )
    fig.update_yaxes(
        title=dict(font=AXIS_LABEL_FONT, standoff=AXIS_LABEL_STANDOFF),
        showline=False,
        showgrid=True, gridcolor=GRID_LINE_COLOR, gridwidth=GRID_LINE_WIDTH,
        zeroline=False, zerolinecolor=GRID_LINE_COLOR, zerolinewidth=ZERO_LINE_WIDTH,
        tickfont=AXIS_TICK_FONT,
        range=[-0.1, len(LABELERS) + 0.02],
    )
    # for r in range(len(sample_metrics)):
    #     fig.update_yaxes(title_text=peyes.constants.LABELER_STR.title(), row=r + 1, col=1)
    fig.update_layout(
        title=dict(
            text=f"Sample-by-Sample Agreement (GT: <i>{gt}</i>)",
            font=TITLE_FONT,
            xref="paper", xanchor="center", x=0.5,
            yref="container", yanchor="bottom", y=0.965,
            automargin=True,
        ),
        width=1200, height=600,
        plot_bgcolor='rgba(0, 0, 0, 0)',
        paper_bgcolor='rgba(0, 0, 0, 0)',
        margin=dict(l=0, r=0, t=50, b=45, pad=0),
        legend=dict(
            font=AXIS_TICK_FONT,
            orientation="h",
            xref="paper", xanchor="center", x=0.5,
            yref="paper", yanchor="bottom", y=0.95,
        ),
    )

# show all figures
for gt, fig in figures.items():
    peyes.visualize.save_figure(
        fig, f"M3_{gt}", FIGURES_DIR, as_png=True, as_json=True
    )
    fig.show()

Sample Metrics :: Global: 100%|██████████| 10/10 [00:01<00:00,  9.20it/s]


### Sample-by-Sample Agreement

#### Statistical Analysis

In [14]:
observations, statistics = dict(), dict()

for c, ds in enumerate(dataset_names):
    dataset = lund_labels2 if c == 0 else hfc_labels2
    ds_short = ds.replace("<b><i>", "").replace("</i></b>", "")
    data = samp.calculate_global_sample_metrics(dataset, GT_ANNOTATORS)
    for r, (metric_key, metric_name) in enumerate(sample_metrics.items()):
        for gt in GT_ANNOTATORS:
            subsets = {
                pred: (
                    data
                    .xs(1, level=peyes.constants.ITERATION_STR, axis=1)
                    .xs(gt, level=u.GT_STR, axis=1)
                    .xs(pred, level=u.PRED_STR, axis=1)
                    .xs(metric_key, axis=0)
                    .map(lambda x: np.round(x, 4))
                ) for pred in LABELERS if pred not in GT_ANNOTATORS
            }
            observations[(ds_short, gt, metric_key)] = subsets
            sub_stats = {
                # "three_way": stats.friedmanchisquare(*subsets.values(), nan_policy="omit"),
                "engbert": stats.wilcoxon(np.round(subsets["remodnav_new"] - subsets["engbert"], 2), alternative="two-sided",),
                "remodnav": stats.wilcoxon(np.round(subsets["remodnav_new"] - subsets["remodnav"], 2), alternative="greater",),
            }
            statistics[(ds_short, gt, metric_key)] = pd.DataFrame({k: {"p": v[1], "statistic": v[0]} for k, v in sub_stats.items()})

# convert `observations` to a DataFrame
sample_level_observations = pd.concat(
    [pd.concat(obs.values(), axis=1, keys=obs.keys()) for obs in observations.values()], keys=observations.keys()
)
sample_level_observations.index.names = ["dataset", u.GT_STR, peyes.constants.METRIC_STR, peyes.constants.TRIAL_ID_STR]

# convert `statistics` to a DataFrame
statistics = pd.concat(statistics).stack(future_stack=True)
statistics.index.names = ["dataset", u.GT_STR, peyes.constants.METRIC_STR, "result_type", "test"]
statistics = statistics.unstack([peyes.constants.METRIC_STR, "result_type"])

statistics

Sample Metrics :: Global: 100%|██████████| 10/10 [00:01<00:00,  9.13it/s]


metric               cohen's_kappa                 mcc            \
result_type                      p statistic         p statistic   
dataset  gt test                                                   
HFC      MN engbert       0.014802       2.0  0.027344       6.0   
            remodnav      0.000977      55.0  0.000977      55.0   
         RA engbert       0.009766       3.0  0.013672       4.0   
            remodnav      0.000977      55.0  0.000977      55.0   
lund2013 MN engbert       0.005697       6.0  0.003052       7.5   
            remodnav      0.000061     105.0  0.000061     105.0   
         RA engbert       0.001150      11.0  0.001659      17.0   
            remodnav      0.000018     203.0  0.000003     208.0   

metric               complement_nld            
result_type                       p statistic  
dataset  gt test                               
HFC      MN engbert        0.046113       2.5  
            remodnav       0.000977      55.0  
         RA engbert        0.070182       8.0  
            remodnav       0.000977      55.0  
lund2013 MN engbert        0.003116       0.0  
            remodnav       0.000061     105.0  
         RA engbert        0.003197       9.0  
            remodnav       0.000002     209.0

#### Sample-Level Comparisons: Engbert vs. REMoDNaV-new

In [15]:
sample_level_pairwise_table = dict()
for ds in sample_level_observations.index.get_level_values("dataset").unique():
    for gt in sample_level_observations.index.get_level_values("gt").unique():
        for metric in sample_level_observations.index.get_level_values("metric").unique():
            remodnav_new = sample_level_observations.loc[(ds, gt, metric), "remodnav_new"].values
            engbert = sample_level_observations.loc[(ds, gt, metric), "engbert"].values
            diff = np.round(remodnav_new - engbert, 3)
            diff_count, diff_median, diff_mean, diff_std = len(diff), np.median(diff), np.mean(diff), np.std(diff)
            W, p = stats.wilcoxon(diff, alternative="two-sided")
            corrected_p = min(1., p * 6)     # Bonferroni correcting for 6 comparisons per dataset (2 annotators, 3 metrics)
            sample_level_pairwise_table[(ds, gt, metric)] = (
                diff_count, diff_median, diff_mean, diff_std / np.sqrt(diff_count), W, p, corrected_p, corrected_p <= 0.05
            )

sample_level_pairwise_table = (
    pd.DataFrame(sample_level_pairwise_table).T
    .rename(columns={0: "N", 1:"median_diff", 2: "mean_diff", 3: "sem_diff", 4:"W", 5:"p", 6:"corrected_p", 7: "is_sig"})
)
sample_level_pairwise_table

C:\Users\jonathanni\AppData\Local\Temp\ipykernel_20096\1518302671.py:5: PerformanceWarning:

indexing past lexsort depth may impact performance.

C:\Users\jonathanni\AppData\Local\Temp\ipykernel_20096\1518302671.py:6: PerformanceWarning:

indexing past lexsort depth may impact performance.



N median_diff mean_diff  sem_diff     W  \
lund2013 RA cohen's_kappa   20       0.035    0.0433  0.010088  15.0   
            mcc             20       0.032    0.0416  0.009597  18.5   
            complement_nld  20       0.013    0.0151  0.004324  20.5   
         MN cohen's_kappa   14       0.066  0.070143   0.01847   7.0   
            mcc             14        0.06  0.064286  0.016744   8.5   
            complement_nld  14       0.015  0.021857  0.005998   6.0   
HFC      RA cohen's_kappa   10      -0.047   -0.0533  0.017089   3.0   
            mcc             10     -0.0365   -0.0434  0.016544   3.0   
            complement_nld  10      -0.009   -0.0079  0.002927   7.0   
         MN cohen's_kappa   10     -0.0355   -0.0486  0.017327   3.0   
            mcc             10      -0.026   -0.0403  0.016688   5.0   
            complement_nld  10      -0.006   -0.0058  0.003078  12.5   

                                   p corrected_p is_sig  
lund2013 RA cohen's_kappa   0.001283    0.007699   True  
            mcc             0.000586    0.003513   True  
            complement_nld  0.002701    0.016206   True  
         MN cohen's_kappa   0.002319    0.013916   True  
            mcc             0.004028     0.02417   True  
            complement_nld  0.001709    0.010254   True  
HFC      RA cohen's_kappa   0.009766    0.058594  False  
            mcc             0.009766    0.058594  False  
            complement_nld  0.037109    0.222656  False  
         MN cohen's_kappa   0.009766    0.058594  False  
            mcc             0.019531    0.117188  False  
            complement_nld  0.160156    0.960938  False

### Boundary Sensitivity

In [16]:
METRIC = peyes.constants.D_PRIME_STR
dataset_names = ["<b><i>lund2013</i></b>", "<b><i>HFC</i></b>"]
boundary_types = [ONSET_STR, OFFSET_STR]
event_types = ["fixation", "saccade"]
figures = {
    (gt, evnt): make_subplots(
        rows=len(boundary_types), cols=len(dataset_names),
        row_titles=list(map(lambda bnd: f"{evnt} {bnd}".title(), boundary_types)), column_titles=dataset_names,
        shared_xaxes=True, shared_yaxes=True,
        horizontal_spacing=0.02, vertical_spacing=0.04,
    ) for gt in GT_ANNOTATORS for evnt in event_types
}

for evnt in event_types:
    for c, ds in enumerate(dataset_names):
        if c == 0:
            dataset = lund_labels2
            plotting_config = lund_plotting_config
        else:
            dataset = hfc_labels2
            plotting_config = hfc_plotting_config
        data = temp.signal_detection_metrics(
            dataset, np.arange(21), ["RA", "MN"], pos_labels=peyes.parse_label(evnt), dprime_correction="loglinear"
        )
        thresholds = data.index.get_level_values(peyes.constants.THRESHOLD_STR).unique().values
        for r, boundary in enumerate(boundary_types):
            for gt in GT_ANNOTATORS:
                curr_fig = figures[(gt, evnt)]
                for pred in LABELERS:
                    if pred == gt:
                        continue  # skip the ground truth annotator
                    if pred in GT_ANNOTATORS:
                        pred_name = f"Ann. {pred}"
                        pred_color = plotting_config["Other Human"][1]
                    elif pred == "remodnav":
                        pred_name = "REMoDNaV"
                        pred_color = plotting_config[pred][1]
                    elif pred == "remodnav_new":
                        pred_name = "REMoDNaV (No SP)"
                        pred_color = "#DA9600"
                    else:
                        pred_name = pred.upper()  # e.g., "engbert"
                        pred_color = plotting_config[pred][1]
                    subset = (
                        data
                        .xs(1, level=peyes.constants.ITERATION_STR, axis=1)
                        .xs(METRIC, level=peyes.constants.METRIC_STR, axis=1)
                        .xs(gt, level=u.GT_STR, axis=1)
                        .xs(pred, level=u.PRED_STR, axis=1)
                        .xs(boundary, axis=0)
                    )
                    if subset.empty:
                        continue
                    curr_fig.add_trace(
                        row=r + 1, col=c + 1,
                        trace=go.Scatter(
                            x=thresholds, y=subset.mean(axis=1).values, error_y=dict(type="data", array=subset.sem(axis=1).values),
                            name=pred_name, legendgroup=pred_name,
                            mode="lines+markers",
                            marker=dict(size=5, color=pred_color),
                            line=dict(color=pred_color, dash="dot" if pred in GT_ANNOTATORS else None),
                            showlegend=(c == 0 and r == 0),
                        )
                    )

# update figure layouts
for (gt, evnt), fig in figures.items():
    for ann in fig.layout.annotations:
        ann.xref = ann.yref = "paper"
        ann.xanchor = "center"
        ann.yanchor = "bottom"
        if ann.text in dataset_names:
            ann.font = SUBTITLE_FONT
            ann.y = 0.975
            continue
        if evnt.title() in ann.text:
            ann.font = AXIS_LABEL_FONT
            ann.textangle = -90
            ann.x = 0.0
            ann.visible=False

    fig.update_xaxes(
        showline=False,
        showgrid=False, gridcolor=GRID_LINE_COLOR, gridwidth=GRID_LINE_WIDTH,
        zeroline=False, zerolinecolor=GRID_LINE_COLOR, zerolinewidth=ZERO_LINE_WIDTH,
    )
    for c in range(len(dataset_names)):
        fig.update_xaxes(
            title=dict(
                text="Δt (<i>samples</i>)",  # alternatively: text="Δt (<i>ms</i>)",
                font=AXIS_LABEL_FONT, standoff=7.5
            ),
            tickfont=AXIS_TICK_FONT, tickmode="array", tickvals=np.arange(0, 21, 5),
            # ticktext=[f"{(2*v):.1f}" for v in np.arange(0, 21, 5)] if c==0 else [f"{(3.33*v):.1f}" for v in np.arange(0, 21, 5)],
            row=len(dataset_names), col=c + 1
        )

    fig.update_yaxes(
        showline=False,
        showgrid=True, gridcolor=GRID_LINE_COLOR, gridwidth=GRID_LINE_WIDTH,
        zeroline=False, zerolinecolor=GRID_LINE_COLOR, zerolinewidth=ZERO_LINE_WIDTH,
        tickfont=AXIS_TICK_FONT,
    )
    for r in range(len(boundary_types)):
        fig.update_yaxes(
            title=dict(
                text=f"{ONSET_STR if r==0 else OFFSET_STR} sensitivity (<i>d'</i>)",
                font=AXIS_LABEL_FONT, standoff=20,
            ),
            row=r + 1, col=1
        )

    fig.update_layout(
        title=dict(
            text=f"{evnt.title()} Boundary Sensitivity (GT: <i>{gt}</i>)",
            font=TITLE_FONT,
            xref="paper", xanchor="center", x=0.5,
            yref="container", yanchor="bottom", y=0.965,
            automargin=True,
        ),
        width=1200, height=600,
        plot_bgcolor='rgba(0, 0, 0, 0)',
        paper_bgcolor='rgba(0, 0, 0, 0)',
        margin=dict(l=0, r=0, t=30, b=0, pad=0),
        legend=dict(
            font=AXIS_TICK_FONT,
            orientation="h",
            xref="paper", xanchor="center", x=0.5,
            yref="paper", yanchor="bottom", y=-0.175,
        ),
    )

# show all figures
for (gt, evnt), fig in figures.items():
    if evnt == "fixation":
        peyes.visualize.save_figure(
            fig, f"M4_{gt}", FIGURES_DIR, as_png=True, as_json=True
        )
        fig.show()

Samples Channel :: SDT Metrics: 100%|██████████| 10/10 [00:03<00:00,  3.06it/s]


#### Statistical Analysis

In [17]:
THRESHOLD = 5       # temporal threshold for analyzing d'
observations, statistics = dict(), dict()

for evnt in event_types:
    if evnt == "saccade":   # TODO: remove this to include saccade analysis
        continue
    for c, ds in enumerate(dataset_names):
        dataset = lund_labels2 if c == 0 else hfc_labels2
        ds_short = ds.replace("<b><i>", "").replace("</i></b>", "")
        data = temp.signal_detection_metrics(
            dataset, np.arange(21), ["RA", "MN"], pos_labels=peyes.parse_label(evnt), dprime_correction="loglinear"
        )
        for gt in GT_ANNOTATORS:
            for bndry in boundary_types:
                subsets = {
                    pred: (
                        data
                        .xs(1, level=peyes.constants.ITERATION_STR, axis=1)
                        .xs(METRIC, level=peyes.constants.METRIC_STR, axis=1)
                        .xs(gt, level=u.GT_STR, axis=1)
                        .xs(pred, level=u.PRED_STR, axis=1)
                        .xs(bndry, axis=0)
                        .loc[THRESHOLD]
                        .map(lambda x: np.round(x, 4))
                    ) for pred in LABELERS if pred not in GT_ANNOTATORS
                }
                observations[(ds_short, gt, evnt, bndry)] = subsets
                sub_stats = {
                    # "three_way": stats.friedmanchisquare(*subsets.values(), nan_policy="omit"),
                    "engbert": stats.wilcoxon(np.round(subsets["remodnav_new"] - subsets["engbert"], 2), alternative="two-sided"),
                    "remodnav": stats.wilcoxon(np.round(subsets["remodnav_new"] - subsets["remodnav"], 2), alternative="greater"),
                }
                statistics[(ds_short, gt, evnt, bndry)] = pd.DataFrame({k: {"p": v[1], "statistic": v[0]} for k, v in sub_stats.items()})

# convert `observations` to a DataFrame
boundary_sensitivity_observations = pd.concat(
    [pd.concat(obs.values(), axis=1, keys=obs.keys()) for obs in observations.values()], keys=observations.keys()
)
boundary_sensitivity_observations.index.names = ["dataset", u.GT_STR, peyes.constants.EVENT_STR, "boundary", peyes.constants.TRIAL_ID_STR]

# # convert `statistics` to a DataFrame
statistics = pd.concat(statistics).stack(future_stack=True)
statistics.index.names = ["dataset", u.GT_STR, peyes.constants.EVENT_STR, "boundary", "result_type", "test"]
statistics = statistics.unstack(["boundary", "result_type"])
statistics = statistics.droplevel(peyes.constants.EVENT_STR)    # TODO: remove this to include saccade analysis

statistics

Samples Channel :: SDT Metrics: 100%|██████████| 10/10 [00:03<00:00,  2.98it/s]


boundary                 onset                  offset          
result_type                  p statistic             p statistic
dataset  gt test                                                
HFC      MN engbert   0.027344       5.5  3.710938e-02       7.0
            remodnav  0.000977      55.0  9.765625e-04      55.0
         RA engbert   0.019531       5.0  1.308594e-01      12.0
            remodnav  0.000977      55.0  9.765625e-04      55.0
lund2013 MN engbert   0.090576      25.0  2.675781e-01      34.0
            remodnav  0.001160      98.0  6.103516e-05     105.0
         RA engbert   0.004221      31.0  6.372261e-02      55.0
            remodnav  0.000052     199.0  9.536743e-07     210.0

In [18]:
boundary_sensitivity_observations

engbert  remodnav  remodnav_new
dataset  gt event    boundary trial_id                                 
lund2013 RA fixation onset    25         3.9972    2.5969        3.0568
                              26         4.8181    2.8972        3.5818
                              27         3.0294    2.1658        2.2791
                              28         3.5012    2.5541        2.6769
                              29         3.7522    2.8831        2.7797
...                                         ...       ...           ...
HFC      MN fixation offset   6          5.0345    2.8101        4.1138
                              7          4.3338    1.9001        2.6205
                              8          3.0680    1.9211        3.0619
                              9          3.7537    2.5838        4.1710
                              10         3.9404    3.4329        4.5178

[108 rows x 3 columns]

#### Fixation Boundary Comparisons: Engbert vs. REMoDNaV-new

In [19]:
boundary_pairwise_table = dict()
for ds in boundary_sensitivity_observations.index.get_level_values("dataset").unique():
    for gt in boundary_sensitivity_observations.index.get_level_values("gt").unique():
        for bndr in boundary_sensitivity_observations.index.get_level_values("boundary").unique():
            remodnav_new = boundary_sensitivity_observations.loc[(ds, gt, "fixation", bndr), "remodnav_new"].values
            engbert = boundary_sensitivity_observations.loc[(ds, gt, "fixation", bndr), "engbert"].values
            diff = np.round(remodnav_new - engbert, 3)
            diff_count, diff_median, diff_mean, diff_std = len(diff), np.median(diff), np.mean(diff), np.std(diff)
            W, p = stats.wilcoxon(diff, alternative="two-sided")
            corrected_p = min(1., p * 4)     # Bonferroni correcting for 4 comparisons per boundary (2 datasets, 2 annotators)
            boundary_pairwise_table[(ds, gt, bndr)] = (
                diff_count, diff_median, diff_mean, diff_std / np.sqrt(diff_count), W, p, corrected_p, corrected_p <= 0.05
            )

boundary_pairwise_table = (
    pd.DataFrame(boundary_pairwise_table).T
    .rename(columns={0: "N", 1:"median_diff", 2: "mean_diff", 3: "sem_diff", 4:"W", 5:"p", 6:"corrected_p", 7: "is_sig"})
    .reorder_levels([2, 0, 1])
    .sort_index(ascending=False)
)
boundary_pairwise_table

C:\Users\jonathanni\AppData\Local\Temp\ipykernel_20096\2450059015.py:5: PerformanceWarning:

indexing past lexsort depth may impact performance.

C:\Users\jonathanni\AppData\Local\Temp\ipykernel_20096\2450059015.py:6: PerformanceWarning:

indexing past lexsort depth may impact performance.



N median_diff mean_diff  sem_diff     W         p  \
onset  lund2013 RA  20     -0.4045  -0.39745  0.113847  32.0   0.00486   
                MN  14     -0.2965 -0.241071  0.118359  25.0  0.090576   
       HFC      RA  10     -0.5275   -0.4861  0.131572   5.0  0.019531   
                MN  10     -0.3805   -0.6067  0.210568   5.0  0.019531   
offset lund2013 RA  20      -0.331  -0.37075  0.168672  55.0  0.063723   
                MN  14      -0.343 -0.338357  0.246112  34.0  0.267578   
       HFC      RA  10      -0.682   -0.4266  0.230973  12.0  0.130859   
                MN  10     -0.6825   -0.4782  0.202758   7.0  0.037109   

                   corrected_p is_sig  
onset  lund2013 RA     0.01944   True  
                MN    0.362305  False  
       HFC      RA    0.078125  False  
                MN    0.078125  False  
offset lund2013 RA     0.25489  False  
                MN         1.0  False  
       HFC      RA    0.523438  False  
                MN    0.148438  False